<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jihoyeo/mobility-simulation-book/blob/main/ko/chapter2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>
<br>


# 2. Data Preparation

본 챕터에서는 모빌리티 시뮬레이션을 구축하는데 필요한 기본적인 입력데이터들을 살펴본다. 단순히 데이터를 살펴보는데서 끝나지 않고 간단한 데이터의 전처리 및 시각화의 실습과정도 포함하고 있다.

매우 다양한 형태의 모빌리티 데이터가 존재하며, 이들을 분류하는 기준 역시 분석자의 목표에 따라 여려가지로 구분할 수 있지만 본 책에서는 크게 아래와 같은 형태의 데이터를 살펴볼 것이다.
1. 통행수요 데이터 (Travel Demand)
    - 시뮬레이션을 하기 위한 승객의 수요를 생성하는 기초자료로 활용됨
    - 수도권 생활이동 데이터 & 스마트카드 데이터
2. 통행시간 데이터 (Travel Time)
    - 시뮬레이션을 Calibration 하는데 활용
    - e.g., 출퇴근 시간의 교통 정체, 새벽시간대의 빠른 속도를 시뮬레이션에 반영 가능함
3. 도로 네트워크 데이터 (Road Network)
    - 도로 위를 움직이는 차량 구현 (Vehicle Router)
4. 대중교통 네트워크 데이터 (Public Transit Network)
    - 버스, 지하철과 같은 노선과 배차간격이 정해진 모빌리티 시스템을 구현하는데 사용